In [32]:
import numpy as np
import requests, gzip, os, hashlib
import scipy 
import pandas as pd 


import cv2

#np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
np.set_printoptions(edgeitems=30, linewidth=100000)


path = './data/mnist'
if not os.path.exists(path):
    os.makedirs(path)

#fetch data
path='./data/mnist'
def fetch(url):
    fp = os.path.join(path, hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp):
        with open(fp, "rb") as f:
            data = f.read()
    else:
        with open(fp, "wb") as f:
            data = requests.get(url).content
            f.write(data)
    return np.frombuffer(gzip.decompress(data), dtype=np.uint8).copy()

X = fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y = fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz")[8:]
X_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz")[0x10:].reshape((-1, 28, 28))
Y_test = fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz")[8:]

#Filter into each number 
X0=[]
X1=[]
X2=[]
X3=[]
X4=[]
X5=[]
X6=[]
X7=[]
X8=[]
X9=[]
for i in range(len(X)):
    if Y[i]==0:
        X0.append(X[i])
    if Y[i]==1:
        X1.append(X[i])
    if Y[i]==2:
        X2.append(X[i])
    if Y[i]==3:
        X3.append(X[i])
    if Y[i]==4:
        X4.append(X[i])
    if Y[i]==5:
        X5.append(X[i])
    if Y[i]==6:
        X6.append(X[i])
    if Y[i]==7:
        X7.append(X[i])
    if Y[i]==8:
        X8.append(X[i])
    if Y[i]==9:
        X9.append(X[i])

In [33]:
def list_of_lists(n):
    """n = number of empty lists you want in list"""
    lofl=[]
    for i in range(n):
        lofl.append([]);
    return lofl

def pixel_list_creator(X):
    """takes all the images and creates 1d arrays of each pixel"""
    pixel_list=list_of_lists(784)
    for i in range(len(X)):
        x=np.reshape(X[i],-1)
        for j in range(len(x)):
            y=x[j]
            pixel_list[j].append(y)
    pixel_list=[np.array(x) for x in pixel_list]
    return pixel_list

def sampling_dist(arr,iterations):
    """iterations: how many samples you want to draw for sampling distribution.
       if you increase the number of samples you decrease variance of distribution."""
    """takes an array and bootstraps a sampling distribution."""
    """returns sampling distribution.  Iterations"""
    distribution=[]
    #change range for higher fidelity sampling distribution, but takes more compute
    for i in range(iterations):
        sample = np.random.choice(arr,size=len(arr),replace=True)
        xbar = np.mean(sample)
        distribution.append(xbar)
    distribution = np.array(distribution)
    return distribution

def sample_mean(arr):
    """returns 50th percentile of sampling distribution"""
    percent_50th=np.percentile(arr,50)
    return percent_50th

def mean_vector(arr_list):
    """takes a list of pixel arrays and returns the mean vector"""
    """for multivariate normal of sampling distributions for each pixel"""
    mu = []
    for i in range(len(arr_list)):
        percentile = sample_mean(arr_list[i])
        mu.append(percentile)
    mu=np.array(mu)
    return mu

def cov_matrix(arr_list):
    """returns the covariance matrix between pixel sampling dristributions"""
    arr_list = [np.sort(x) for x in arr_list]
    x = np.vstack([arr_list])
    cov=np.cov(x)
    return cov

def mean_cov(X,iterations):
    """Iterations: see sampling_dist() function above for details"""
    """Returns mean vector and covariance matrix for bootstrpped multivariate normal"""
    pixel_list = pixel_list_creator(X)
    sampling_dist_list = [sampling_dist(x,iterations) for x in pixel_list]
    mu_vector = mean_vector(sampling_dist_list)
    cov = cov_matrix(sampling_dist_list)
    return mu_vector, cov

def multivariate_sample(mu,covariance):
    """extracts random sample from multivariate distribution defined by specific mean vector"""
    """and covariance matrix."""
    x=np.random.multivariate_normal(mu,covariance)
    x=np.rint(x)
    x=x.astype(int)
    x=np.reshape(x,(28,28))
    return x

def final_xbar(mu, covariance, sample_size):
    """repeatedly samples defined multivariate normal distribution and provides mean of all samples"""
    test=multivariate_sample(mu,covariance)
    for i in range(sample_size):
        x = multivariate_sample(mu,covariance)
        test=test+x
    test=test/(sample_size+1)
    test=np.rint(test)
    test=test.astype(int)
    return test

## Zero

### Average over samples

In [34]:
test_zero=mean_cov(X0,1000)
mu=test_zero[0]
covariance=test_zero[1]
final_zero=final_xbar(mu, covariance,10000)
print(final_zero)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   1   2   3   4   5   5   4   3   2   2   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   1   4   7  14  25  39  53  63  66  61  50  36  22  11   4   1   0   0   0   0]
 [  0   0   0   0   0   0   1   2   5  12  24  44  69  99 126 144 150 141 118  89  57  32  13   4   1   0   0   0]
 [  0   0   0   0   0   1   2   5  13  28  52  83 117 149 173 188 192 188 171 139 101  62  31  10   1   0   0   0]
 [  0   0   0   0   0   1   3  11  26  52  85 122 152 173 185 188 190 192 189 171 138  95  54  21   4   0   0   0]
 [  0   0   0   0   0   2   7  21  46  80 119 151 169 173 167 158 154 158 169 17

### Individual Samples

In [62]:
test1=multivariate_sample(test_zero[0],test_zero[1])
print(test1)
test2=multivariate_sample(test_zero[0],test_zero[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   1   2   3   4   5   5   4   3   2   2   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   1   4   7  14  25  39  53  63  67  61  50  36  22  11   4   1   0   0   0   0]
 [  0   0   0   0   0   0   1   2   5  12  24  44  70  99 126 144 150 141 119  89  57  32  13   4   1   0   0   0]
 [  0   0   0   0   0   1   2   5  13  28  52  83 117 149 173 188 192 188 171 139 101  62  31  10   1   0   0   0]
 [  0   0   0   0   0   1   3  11  26  52  85 122 153 173 185 188 190 192 189 171 138  95  54  21   4   0   0   0]
 [  0   0   0   0   0   2   7  21  46  80 119 151 169 173 167 158 154 158 169 17

## One

### Average over samples

In [35]:
test_one=mean_cov(X1,1000)
mu=test_one[0]
covariance=test_one[1]
final_one=final_xbar(mu, covariance, 10000)
print(final_one)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   2   2   1   1   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   2   5  14  27  39  45  44  41  35  28  19  10   4   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   4  11  27  52  76  89  89  81  68  52  34  18   7   3   1   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   4  12  31  63  94 112 112 100  82  57  35  17   7   2   1   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   4  11  32  68 106 129 128 109  81  52  28  12   4   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   3  10  30  72 120 150 142 112  74  4

### Individual Samples

In [63]:
test1=multivariate_sample(test_one[0],test_one[1])
print(test1)
test2=multivariate_sample(test_one[0],test_one[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   2   1   1   1   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   2   5  14  27  39  44  43  41  35  28  18   9   4   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   3  10  26  51  75  88  88  80  67  51  33  17   7   2   1   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   4  12  30  62  93 111 111  99  80  56  34  16   6   2   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   3  11  31  67 105 128 127 108  80  51  27  12   4   1   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   3  10  30  71 119 149 141 111  73  4

## Two

### Average over samples

In [36]:
test_two=mean_cov(X2,1000)
mu=test_two[0]
covariance=test_two[1]
final_two=final_xbar(mu, covariance, 10000)
print(final_two)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   2   3   3   4   4   4   3   2   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   3   6  12  18  26  33  39  41  41  36  28  18  11   5   2   1   0   0   0   0   0]
 [  0   0   0   0   0   2   5  11  21  36  54  75  95 110 118 116 107  86  62  41  23  10   3   0   0   0   0   0]
 [  0   0   0   0   1   4  12  25  44  67  93 118 140 156 164 165 155 135 104  72  44  22   9   2   0   0   0   0]
 [  0   0   0   0   2   7  19  39  62  87 111 132 145 154 159 162 160 149 126  94  62  35  14   3   0   0   0   0]
 [  0   0   0   0   3  10  25  45  66  87 104 116 120 121 123 128 136 139 130 106  74  44  20   5   0   0   0   0]
 [  0   0   0   0   3  11  26  42  58  71  81  85  84  81  83  93 110 127 130 11

### Individual samples

In [64]:
test1=multivariate_sample(test_two[0],test_two[1])
print(test1)
test2=multivariate_sample(test_two[0],test_two[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   2   3   3   4   4   4   3   2   1   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   3   6  12  18  26  33  39  42  41  36  28  19  11   6   2   1   0   0   0   0   0]
 [  0   0   0   0   0   2   5  11  21  37  54  75  95 111 118 117 107  86  63  41  23  11   3   1   0   0   0   0]
 [  0   0   0   0   1   4  12  25  44  68  93 119 140 156 165 166 156 136 105  72  45  23   9   2   0   0   0   0]
 [  0   0   0   0   2   8  20  39  62  88 112 133 146 154 159 162 161 150 127  95  63  35  15   3   0   0   0   0]
 [  0   0   0   0   3  10  26  45  67  87 104 116 121 122 124 129 136 140 131 106  75  45  21   5   0   0   0   0]
 [  0   0   0   0   3  12  27  43  59  72  82  85  84  82  84  93 111 128 130 11

## Three

### Average over samples

In [37]:
test_three=mean_cov(X3,1000)
mu=test_three[0]
covariance=test_three[1]
final_three=final_xbar(mu, covariance, 10000)
print(final_three)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1   2   4   5   6   7   6   5   4   3   2   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   1   3   8  14  23  35  47  56  63  64  60  51  38  27  17   8   3   1   0   0   0   0   0]
 [  0   0   0   0   2   7  16  29  48  72  98 122 143 156 159 152 135 107  78  52  30  14   4   1   0   0   0   0]
 [  0   0   0   1   6  15  31  53  81 111 139 160 175 185 190 190 180 155 122  87  54  28  10   2   0   0   0   0]
 [  0   0   0   2  10  21  39  62  86 109 125 135 139 142 150 160 166 158 135 104  69  37  14   3   0   0   0   0]
 [  0   0   0   3  10  20  34  50  64  75  81  81  79  82  93 115 137 146 134 10

### Individual samples

In [65]:
test1=multivariate_sample(test_three[0],test_three[1])
print(test1)
test2=multivariate_sample(test_three[0],test_three[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   2   3   4   6   7   7   7   6   5   3   2   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   2   4   8  15  24  36  48  57  64  65  61  52  39  28  17   9   4   1   0   0   0   0   0]
 [  0   0   0   0   2   8  16  30  49  73  99 124 144 158 160 154 136 109  79  53  31  14   5   1   0   0   0   0]
 [  0   0   0   2   7  16  32  54  83 112 140 161 176 186 191 191 181 156 124  88  56  29  10   2   0   0   0   0]
 [  0   0   0   3  10  22  40  63  87 110 127 136 140 144 151 161 167 159 136 105  70  38  14   3   0   0   0   0]
 [  0   0   0   3  11  21  35  51  65  76  82  82  81  83  95 117 138 147 135 10

## Four

### Average over samples

In [38]:
test_four=mean_cov(X4,1000)
mu=test_four[0]
covariance=test_four[1]
final_four=final_xbar(mu, covariance, 10000)
print(final_four)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   1   2   1   1   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   1   2   3   4   4   4   4   4   6   8  10  12  13  11   8   5   2   0   0   0]
 [  0   0   0   0   0   1   3   5   8  13  16  19  19  17  16  16  22  30  38  40  38  32  25  17   7   1   0   0]
 [  0   0   0   0   1   3   7  12  20  30  39  43  42  36  28  27  37  53  66  69  62  51  38  25  11   2   0   0]
 [  0   0   0   0   2   5  11  20  34  50  63  67  63  51  38  35  48  71  90  92  79  61  43  25  10   2   0   0]
 [  0   0   0   1   3   7  14  28  47  71  86  89  80  62  42  36  54  86 110 10

### Individual samples

In [66]:
test1=multivariate_sample(test_four[0],test_four[1])
print(test1)
test2=multivariate_sample(test_four[0],test_four[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   1   2   1   1   1   0   0   0   0]
 [  0   0   0   0   0   0   0   1   2   2   3   4   4   4   4   4   6   8  10  12  13  11   8   5   2   0   0   0]
 [  0   0   0   0   1   2   3   5   8  13  17  19  20  18  16  16  22  31  38  40  39  33  26  17   7   1   0   0]
 [  0   0   0   0   1   3   7  13  20  31  40  43  42  36  28  28  37  53  67  70  63  52  39  25  11   2   0   0]
 [  0   0   0   0   2   6  11  20  34  51  64  68  63  52  38  35  48  71  90  92  79  62  43  25  10   2   0   0]
 [  0   0   0   1   3   7  15  29  48  71  87  90  80  62  42  37  55  87 110 10

## Five

### Average over samples

In [39]:
test_five=mean_cov(X5,1000)
mu=test_five[0]
covariance=test_five[1]
final_five=final_xbar(mu, covariance, 10000)
print(final_five)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   1   2   3   3   3   3   3   3   2   1   1   0   0   0   0]
 [  0   0   0   0   0   0   0   1   1   3   6  10  13  17  21  25  27  28  26  24  21  18  14  10   5   1   0   0]
 [  0   0   0   0   0   0   2   4   8  15  26  38  50  62  71  77  80  82  79  74  66  57  46  33  19   6   1   0]
 [  0   0   0   0   0   1   4   9  18  32  52  73  94 110 122 126 127 127 123 114 103  89  74  56  35  14   3   0]
 [  0   0   0   0   0   2   6  15  29  50  76 103 123 136 141 140 138 135 129 120 108  95  81  66  45  21   4   0]
 [  0   0   0   0   0   2   8  20  39  65  93 119 133 135 131 124 117 112 105  9

### Individual samples

In [67]:
test1=multivariate_sample(test_five[0],test_five[1])
print(test1)
test2=multivariate_sample(test_five[0],test_five[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   1   1   2   3   3   3   3   3   3   2   2   1   0   0   0   0]
 [  0   0   0   0   0   0   0   1   2   3   7  10  13  17  21  25  28  28  27  24  21  19  15  10   5   1   0   0]
 [  0   0   0   0   0   0   2   4   8  16  26  38  51  62  71  78  81  82  79  74  67  58  47  33  19   6   1   0]
 [  0   0   0   0   0   1   4   9  18  33  52  74  94 111 122 127 128 127 123 115 103  89  74  57  36  15   3   0]
 [  0   0   0   0   0   2   6  15  29  50  77 103 124 136 141 141 139 136 129 120 109  96  82  67  45  21   4   0]
 [  0   0   0   0   0   3   8  20  40  65  94 120 134 136 132 124 118 112 105  9

## Six

### Average over samples

In [40]:
test_six=mean_cov(X6,1000)
mu=test_six[0]
covariance=test_six[1]
final_six=final_xbar(mu, covariance, 10000)
print(final_six)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   1   1   2   2   2   1   1   1   1   1   1   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   2   4   7  10  16  22  28  32  34  32  27  20  12   6   3   1   0   0   0   0]
 [  0   0   0   0   0   0   1   2   4   9  15  23  33  47  63  76  81  78  67  49  30  16   7   3   1   0   0   0]
 [  0   0   0   0   0   1   1   3   7  14  23  34  49  70  90 104 108  99  80  56  33  18   8   3   1   0   0   0]
 [  0   0   0   0   0   1   2   5  10  19  31  46  68  91 110 117 111  92  67  43  24  13   6   3   1   0   0   0]
 [  0   0   0   0   0   1   2   7  14  26  40  62  88 109 121 117  96  69  43  24  13   7   4   2   1   0   0   0]
 [  0   0   0   0   0   1   3   9  19  34  54  80 106 123 123 103  72  43  22  11   6   4   2   1   1   0   0   0]
 [  0   0   0   0   0   1   4  12  26  44  70  99 122 129 114  80  46  22  10   

### Individual samples

In [68]:
test1=multivariate_sample(test_six[0],test_six[1])
print(test1)
test2=multivariate_sample(test_six[0],test_six[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   1   1   1   1   1   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   2   4   6  10  15  21  27  31  33  31  26  19  11   6   3   1   0   0   0   0]
 [  0   0   0   0   0   0   1   2   4   8  14  22  32  45  62  75  79  76  65  48  29  15   7   2   1   0   0   0]
 [  0   0   0   0   0   0   1   3   7  13  22  33  48  69  89 103 106  97  79  55  32  17   8   3   1   0   0   0]
 [  0   0   0   0   0   1   2   4  10  18  30  44  67  90 108 116 110  91  66  42  23  12   6   3   1   0   0   0]
 [  0   0   0   0   0   1   2   6  13  25  39  61  87 108 119 115  95  68  42  23  12   6   3   2   1   0   0   0]
 [  0   0   0   0   0   1   3   8  18  33  52  79 104 121 121 102  71  42  21  11   6   4   2   1   0   0   0   0]
 [  0   0   0   0   0   1   4  12  25  43  68  98 120 128 112  79  45  21  10   

## Seven

### Average over samples

In [41]:
test_seven=mean_cov(X7,1000)
mu=test_seven[0]
covariance=test_seven[1]
final_seven=final_xbar(mu, covariance, 10000)
print(final_seven)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   1   2   2   2   2   3   2   2   2   2   1   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   3   6   9  14  18  23  25  27  28  28  28  27  25  20  15   9   6   3   1   0   0   0   0]
 [  0   0   1   3   7  15  27  40  55  70  83  93 100 105 108 111 112 108  96  77  54  31  15   6   1   0   0   0]
 [  0   0   3   9  18  34  54  79 105 130 151 166 175 180 183 185 186 182 166 13

### Individual samples

In [69]:
test1=multivariate_sample(test_seven[0],test_seven[1])
print(test1)
test2=multivariate_sample(test_seven[0],test_seven[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   1   1   2   2   2   2   2   2   2   2   1   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   1   3   6   9  13  18  22  25  27  27  28  27  26  24  20  14   9   6   3   1   0   0   0   0]
 [  0   0   1   3   7  15  26  39  54  69  82  92  99 104 107 110 111 107  95  77  53  30  14   6   1   0   0   0]
 [  0   0   3   8  18  33  53  78 104 129 150 165 175 179 182 184 185 181 165 13

## Eight

### Average over samples

In [42]:
test_eight=mean_cov(X8,1000)
mu=test_eight[0]
covariance=test_eight[1]
final_eight=final_xbar(mu, covariance, 10000)
print(final_eight)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   2   4   7  13  21  28  34  34  30  23  17  11   6   3   1   1   0   0   0]
 [  0   0   0   0   0   0   1   2   5  12  23  42  67  95 119 134 138 128 105  78  52  32  17   9   4   1   0   0]
 [  0   0   0   0   0   1   3   8  18  35  60  95 130 159 178 186 185 175 154 124  91  60  36  20   9   3   1   0]
 [  0   0   0   0   1   2   7  18  36  63  97 132 158 168 165 160 155 153 151 136 110  79  51  30  14   4   1   0]
 [  0   0   0   0   1   5  13  28  52  85 120 144 152 138 115 101 100 111 128 13

### Individual smaples

In [70]:
test1=multivariate_sample(test_eight[0],test_eight[1])
print(test1)
test2=multivariate_sample(test_eight[0],test_eight[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   1   1   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   1   2   4   8  13  21  29  34  35  31  24  17  11   7   3   1   1   0   0   0]
 [  0   0   0   0   0   0   1   2   6  12  23  42  68  96 120 135 138 128 106  79  53  32  18   9   4   1   0   0]
 [  0   0   0   0   0   1   3   8  18  35  61  95 131 160 179 187 186 176 154 125  92  61  37  20   9   3   1   0]
 [  0   0   0   0   1   2   7  18  37  63  98 133 159 169 166 161 156 154 151 136 111  80  52  30  14   5   1   0]
 [  0   0   0   0   1   5  13  29  52  85 121 145 153 139 116 102 101 112 129 13

## Nine

### Average over smaples

In [43]:
test_nine=mean_cov(X9,1000)
mu=test_nine[0]
covariance=test_nine[1]
final_nine=final_xbar(mu, covariance, 10000)
print(final_nine)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   1   2   4   7   9  11  11  10   8   5   3   2   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   3   8  16  31  52  77  99 110 105  87  62  37  19   8   2   1   0   0   0   0]
 [  0   0   0   0   0   0   2   5  13  28  54  92 134 170 193 199 189 160 120  78  43  20   7   2   0   0   0   0]
 [  0   0   0   0   0   1   5  14  33  64 109 153 181 188 180 173 170 161 139 10

### Individual samples

In [71]:
test1=multivariate_sample(test_nine[0],test_nine[1])
print(test1)
test2=multivariate_sample(test_nine[0],test_nine[1])
print(test2)
#test to make sure that there is variation amongst the samples
print("\nNumber of matching pixels: "+str(np.sum(test1==test2)))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   1   1   0   0   0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   2   4   6   9  10  11   9   7   5   3   2   1   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   1   3   7  16  30  51  76  98 109 104  86  61  36  18   7   2   0   0   0   0   0]
 [  0   0   0   0   0   0   1   5  13  28  54  91 133 169 192 198 188 159 119  77  42  20   7   2   0   0   0   0]
 [  0   0   0   0   0   1   5  14  33  63 108 152 180 187 179 172 169 160 138  9